In [19]:
import os
import numpy as np
import pandas as pd
import glob

# Get all directories in the current path starting with a digit (representing model folders)
model_dirs = sorted([d for d in os.listdir('.') if os.path.isdir(d) and d[0].isdigit()])

print(f"{'Folder Name':<30} | {'MAE (meV/atom)':<20}| {'RMSE (meV/atom)':<20}")
print("-" * 75)

for model_dir in model_dirs:
    
    # Search for *.pckl.gzip files within the current model directory
    pckl_files = glob.glob(os.path.join(model_dir, "*.pckl.gzip"))
    
    all_absolute_errors = []
    all_squared_errors = []
    
    for pred_path in pckl_files:
        pred_filename = os.path.basename(pred_path)
        
        # Reconstruct the corresponding DFT filename
        # Remove '.pckl_new' to match the original DFT file convention
        dft_filename = pred_filename.replace('.pckl_new', '')
        dft_path = os.path.join("DFT", dft_filename)
        
        # Check if the corresponding DFT file exists
        if not os.path.exists(dft_path):
            print(f"Warning: DFT file not found for {pred_filename}")
            continue
            
        try:
            df_pred = pd.read_pickle(pred_path, compression="gzip")
            df_true = pd.read_pickle(dft_path, compression="gzip")
        except Exception as e:
            print(f"Error reading {pred_filename}: {e}")
            continue
        
        # Ensure data length consistency
        if len(df_pred) != len(df_true):
            print(f"Length mismatch in {pred_filename}")
            continue

        # Iterate through paired rows to calculate errors
        for (idx, row_pred), (_, row_true) in zip(df_pred.iterrows(), df_true.iterrows()):
            
            # Get the number of atoms
            atoms = row_pred['ase_atoms']
            n_atoms = len(atoms)
            
            if n_atoms == 0:
                continue
            
            # --- Core Calculation Logic Start ---
            
            # 1. UMLIP Prediction: Already in eV/atom, take directly
            e_pred_per_atom = row_pred['energy']
            
            # 2. DFT Ground Truth: Total energy (eV), needs normalization by N_atoms
            e_true_per_atom = row_true['energy'] / n_atoms
            
            # 3. Calculate Error (convert eV/atom -> meV/atom)
            # Formula: (Predicted - True) * 1000
            error_per_atom_mev = (e_pred_per_atom - e_true_per_atom) * 1000
            
            # --- Core Calculation Logic End ---
            
            all_absolute_errors.append(abs(error_per_atom_mev))
            all_squared_errors.append(error_per_atom_mev ** 2)
            
    # Calculate and print statistical metrics (MAE and RMSE) for the current model
    if all_absolute_errors:
        mae = sum(all_absolute_errors) / len(all_absolute_errors)
        rmse = np.sqrt(sum(all_squared_errors) / len(all_squared_errors))
        print(f"{model_dir:<30} | {mae:.4f}               | {rmse:.4f}")
    else:
        print(f"{model_dir:<30} | No Data              | No Data")

Folder Name                    | MAE (meV/atom)      | RMSE (meV/atom)     
---------------------------------------------------------------------------
0-GRACE-2L-OMAT                | 13.6463               | 50.4278
1-GRACE-2L-OAM                 | 66.4454               | 97.8056
10-eqV2-31-omat                | 8.1553               | 48.8050
11-eqV2-86-omat                | 7.2176               | 49.0241
12-eqV2-153-omat               | 6.8205               | 48.7410
13-esen-omat                   | 7.2268               | 49.2141
14-orb-omat-c-inf-conf         | 8.4308               | 48.3675
15-orb-omat-20                 | 8.5560               | 48.7295
16-orb-omat-d-inf              | 8.4654               | 47.1899
17-orb-omat-d-20               | 8.5808               | 45.7772
18-7net-omat                   | 11.1827               | 50.4531
19-7net-omat-2                 | 11.8134               | 50.3619
2-eqV2-31                      | 64.1742               | 95.1208
20-MACE-mpa

In [22]:
import os
import numpy as np
import pandas as pd
import glob
import math

# Get all directories in the current path starting with a digit 
# (These represent the different model folders)
model_dirs = sorted([d for d in os.listdir('.') if os.path.isdir(d) and d[0].isdigit()])

print(f"{'Folder Name':<30} | {'Force RMSE (eV/A)':<20}")
print("-" * 55)

for model_dir in model_dirs:
    
    # Search for .pckl.gzip files within the current model directory
    pckl_files = glob.glob(os.path.join(model_dir, "*.pckl.gzip"))
    
    # Initialize accumulators for RMSE calculation
    total_squared_error = 0.0  # Sum of squared errors
    total_components = 0       # Total number of valid components (n_atoms * 3)
    
    for pred_path in pckl_files:
        pred_filename = os.path.basename(pred_path)
        
        # Reconstruct the corresponding DFT filename
        dft_filename = pred_filename.replace('.pckl_new', '')
        dft_path = os.path.join("DFT", dft_filename)
        
        # Check if the corresponding DFT file exists
        if not os.path.exists(dft_path):
            print(f"Warning: DFT file not found for {pred_filename}")
            continue
            
        try:
            df_pred = pd.read_pickle(pred_path, compression="gzip")
            df_true = pd.read_pickle(dft_path, compression="gzip")
        except Exception as e:
            print(f"Error reading {pred_filename}: {e}")
            continue
        
        # Ensure data length consistency
        if len(df_pred) != len(df_true):
            print(f"Length mismatch in {pred_filename}")
            continue

        # Check if 'forces' column exists, otherwise try ASE getter
        if 'forces' not in df_pred.columns or 'forces' not in df_true.columns:
            try:
                 # Attempt to access forces via ASE atoms object
                 _ = df_pred.iloc[0]['ase_atoms'].get_forces()
                 use_ase_getter = True
            except:
                 print(f"Skipping {pred_filename}: No 'forces' column or calculator found.")
                 continue
        else:
            use_ase_getter = False

        # Iterate through each row (configuration)
        for (idx, row_pred), (_, row_true) in zip(df_pred.iterrows(), df_true.iterrows()):
            
            # Extract forces
            if use_ase_getter:
                f_pred = row_pred['ase_atoms'].get_forces()
                f_true = row_true['ase_atoms'].get_forces()
            else:
                f_pred = row_pred['forces']
                f_true = row_true['forces']
            
            # Convert to numpy arrays
            f_pred = np.array(f_pred)
            f_true = np.array(f_true)
            
            # Shape check
            if f_pred.shape != f_true.shape:
                continue
            
            # --- Updated Logic: NaN Masking & Calculation ---
            
            # 1. Flatten the arrays to 1D. 
            # This allows us to handle x, y, z components uniformly and apply the mask easily.
            f_pred_flat = f_pred.ravel()
            f_true_flat = f_true.ravel()
            
            # 2. Create a mask to filter out NaN values
            # We check both predicted and true arrays for safety.
            mask = ~np.isnan(f_pred_flat) & ~np.isnan(f_true_flat)
            
            # 3. Apply the mask to keep only valid data points
            f_pred_clean = f_pred_flat[mask]
            f_true_clean = f_true_flat[mask]
            
            if f_pred_clean.size == 0:
                continue
                
            # 4. Calculate squared difference for components
            # Sum((Pred - True)^2)
            # This computes the squared norm of the difference vector component-wise
            diff_sq = (f_pred_clean - f_true_clean) ** 2
            
            # 5. Accumulate results
            total_squared_error += np.sum(diff_sq)
            total_components += f_pred_clean.size
            
    # Calculate global RMSE for the current folder
    if total_components > 0:
        mse = total_squared_error / total_components
        rmse = math.sqrt(mse)
        print(f"{model_dir:<30} | {rmse:.4f}")
    else:
        print(f"{model_dir:<30} | No Data")

Folder Name                    | Force RMSE (eV/A)   
-------------------------------------------------------
0-GRACE-2L-OMAT                | 0.0862
1-GRACE-2L-OAM                 | 0.2294
10-eqV2-31-omat                | 0.0515
11-eqV2-86-omat                | 0.0462
12-eqV2-153-omat               | 0.0448
13-esen-omat                   | 0.0507
14-orb-omat-c-inf-conf         | 0.0781
15-orb-omat-20                 | 0.0792
16-orb-omat-d-inf              | 0.0704
17-orb-omat-d-20               | 0.0732
18-7net-omat                   | 0.0880
19-7net-omat-2                 | 0.0799
2-eqV2-31                      | 0.0934
20-MACE-mpa                    | 0.3291
3-eqV2-86                      | 0.0886
4-eqV2-153                     | 0.1306
5-DPA3-v1                      | 0.2312
6-DPA3-V2                      | 0.3360
7-7net                         | 0.0966
8-esen                         | 0.1562
9-MACE-omat                    | 0.0906
